In [0]:
import shutil

shutil.rmtree(
    "/Volumes/ecommerce/silver/checkpoint/order_enrich_checkpoint/",
    ignore_errors=True
)
shutil.rmtree(
    "/Volumes/ecommerce/silver/checkpoint/order_item_enrich_checkpoint/",
    ignore_errors=True
)

df_cust = spark.sql("select * from workspace.default.dim_customers")
df_prod = spark.sql("select * from workspace.default.dim_products")
df_orders = spark.readStream.table("ecommerce.bronze.brz_orders")
df_order_itmes = spark.readStream.table("ecommerce.bronze.brz_order_items")


slv_order_enrich = df_orders.join(df_cust,df_cust.customer_id == df_orders.customer_id,"inner")
slv_order_item_enrich = df_order_itmes.join(df_prod,df_prod.product_id == df_order_itmes.product_id,"inner")

slv_order_enrich = slv_order_enrich.drop("customer_id")
slv_order_item_enrich = slv_order_item_enrich.drop("product_id")

slv_order_enrich.writeStream\
    .format("delta") \
    .option("checkpointLocation","/Volumes/ecommerce/silver/checkpoint/order_enrich_checkpoint/")\
    .option("mergeSchema", "true")\
    .option("mode","append")\
    .trigger(once = True)\
    .table("ecommerce.silver.slv_order_enrich")\
    .awaitTermination()

slv_order_item_enrich.writeStream\
    .format("delta")\
    .option("checkpointLocation","/Volumes/ecommerce/silver/checkpoint/order_item_enrich_checkpoint/")\
    .option("mergeSchema", "true")\
    .option("mode","append")\
    .trigger(once = True)\
    .table("ecommerce.silver.slv_order_item_enrich")\
    .awaitTermination()


In [0]:
%sql
select * from ecommerce.silver.slv_order_enrich

In [0]:
%sql
select * from ecommerce.silver.slv_order_item_enrich